In [1]:
from mainnet_launch.gas_costs.keeper_network_gas_costs import (
    fetch_solver_gas_costs,
    fetch_keeper_network_gas_costs,
    fetch_all_autopool_debt_reporting_events,
)

destination_debt_reporting_df = fetch_all_autopool_debt_reporting_events()
rebalance_gas_cost_df = fetch_solver_gas_costs()
keeper_gas_costs_df = fetch_keeper_network_gas_costs()  # 2 min

2024-10-29 14:56:56.927 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:56:56.930 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:56:56.932 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:56:56.933 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:56:56.934 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:56:56.936 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-29 14:57:06.040 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:57:06.220 
  command:

    streamlit run /home/park

In [13]:
import pandas as pd

gas_cost_columns = ["hash", "gas_cost_in_eth"]

debt_reporting_costs = destination_debt_reporting_df[gas_cost_columns].copy().drop_duplicates()
debt_reporting_costs.columns = ["hash", "debt_reporting_gas"]

solver_costs = rebalance_gas_cost_df[gas_cost_columns].copy().drop_duplicates()
solver_costs.columns = ["hash", "solver_gas"]

keeper_costs = keeper_gas_costs_df[gas_cost_columns].copy().drop_duplicates()
keeper_costs.columns = ["hash", "keeper_gas"]

gas_cost_df = pd.concat([debt_reporting_costs, solver_costs, keeper_costs])

lifetime_costs = gas_cost_df[["debt_reporting_gas", "solver_gas", "keeper_gas"]].sum().to_dict()

{'debt_reporting_gas': 8.955499179256435,
 'solver_gas': 13.315402236781724,
 'keeper_gas': 43.13051226866139}

In [4]:
from mainnet_launch.autopool_diagnostics.fees import fetch_autopool_fee_data
import pandas as pd
import plotly.express as px
from mainnet_launch.constants import (
    CACHE_TIME,
    eth_client,
    ALL_AUTOPOOLS,
    BAL_ETH,
    AUTO_ETH,
    AUTO_LRT,
    AutopoolConstants,
    WORKING_DATA_DIR,
)
from datetime import datetime, timedelta, timezone


def compute_revenue_from_fees() -> dict[str, dict[str, float]]:
    """
    Fetch all the periodic and streaming fee events for each autopool then sum the ETH
    value from the `fees` event by 7 day, 30 day and 1 year. then sum those value
    The data should look like

    {
        '7_days': {
            'autoETH_periodic': 1.671720333158365,
            'autoETH_streaming': 3.1315890919246083,
            'balETH_periodic': 0.7185848926255698,
            'balETH_streaming': 0.8942176501922965,
            'autoLRT_periodic': 0.4636031290053162,
            'autoLRT_streaming': 0.43119494960570615,
            'total_revenue': 7.310910046511863
        },
        '30_days': {
            'autoETH_periodic': 6.691773228438292,
            'autoETH_streaming': 11.565605559895612,
            'balETH_periodic': 3.039947675877742,
            'balETH_streaming': 4.815447767226244,
            'autoLRT_periodic': 1.9713912472088526,
            'autoLRT_streaming': 2.280668705081137,
            'total_revenue': 30.36483418372788
        },
        '1_year': {
            'autoETH_periodic': 8.596291557174354,
            'autoETH_streaming': 16.335460959034297,
            'balETH_periodic': 3.9371002057270115,
            'balETH_streaming': 7.222694296056844,
            'autoLRT_periodic': 2.5558998672512345,
            'autoLRT_streaming': 3.6563966618798514,
            'total_revenue': 42.30384354712359
        }
    }

    """
    fee_dfs = []
    for autopool in ALL_AUTOPOOLS:
        periodic_fee_df, streaming_fee_df = fetch_autopool_fee_data(autopool)
        periodic_fee_df.columns = [f"{autopool.name}_periodic"]
        streaming_fee_df.columns = [f"{autopool.name}_streaming"]
        fee_dfs.extend([periodic_fee_df, streaming_fee_df])

    fee_df = pd.concat(fee_dfs).fillna(0)

    today = datetime.now(timezone.utc)
    seven_days_ago = today - timedelta(days=7)
    thirty_days_ago = today - timedelta(days=30)
    year_ago = today - timedelta(days=365)

    revenue_in_last_7_days = fee_df[fee_df.index >= seven_days_ago].sum().to_dict()
    revenue_in_last_7_days["total_revenue"] = sum([revenue_in_last_7_days[k] for k in fee_df.columns])

    revenue_in_last_30_days = fee_df[fee_df.index >= thirty_days_ago].sum().to_dict()
    revenue_in_last_30_days["total_revenue"] = sum([revenue_in_last_30_days[k] for k in fee_df.columns])

    revenue_in_last_year = fee_df[fee_df.index >= year_ago].sum().to_dict()
    revenue_in_last_year["total_revenue"] = sum([revenue_in_last_year[k] for k in fee_df.columns])

    return {"7_days": revenue_in_last_7_days, "30_days": revenue_in_last_30_days, "1_year": revenue_in_last_year}


revenue_data = compute_revenue_from_fees()
revenue_data

2024-10-29 14:53:52.990 No runtime found, using MemoryCacheStorageManager
2024-10-29 14:53:52.991 No runtime found, using MemoryCacheStorageManager
2024-10-29 14:53:52.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:52.993 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:52.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:52.995 No runtime found, using MemoryCacheStorageManager
2024-10-29 14:53:53.494 Thread 'Thread-16': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:53.495 Thread 'Thread-16': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:55.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 14:53:55.013 

{'7_days': {'autoETH_periodic': 1.6299798785501705,
  'autoETH_streaming': 3.1315890919246083,
  'balETH_periodic': 0.6994237991888523,
  'balETH_streaming': 0.8942176501922965,
  'autoLRT_periodic': 0.45253066284501975,
  'autoLRT_streaming': 0.43119494960570615,
  'total_revenue': 7.238936032306654},
 '30_days': {'autoETH_periodic': 6.767933447749041,
  'autoETH_streaming': 11.565605559895612,
  'balETH_periodic': 3.0716906015318104,
  'balETH_streaming': 4.815447767226244,
  'autoLRT_periodic': 1.9922007254892007,
  'autoLRT_streaming': 2.280668705081137,
  'total_revenue': 30.493546806973043},
 '1_year': {'autoETH_periodic': 8.6724517764851,
  'autoETH_streaming': 16.3354609590343,
  'balETH_periodic': 3.96884313138108,
  'balETH_streaming': 7.222694296056844,
  'autoLRT_periodic': 2.576709345531583,
  'autoLRT_streaming': 3.6563966618798522,
  'total_revenue': 42.43255617036876}}

In [ ]:
# total costs at 62ETH and total revenue at 42, 21 if reducing for acctoke

In [5]:
# destination_debt_reporting_df

In [6]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [8]:
keeper_gas_costs_df.columns, destination_debt_reporting_df.columns, rebalance_gas_cost_df.columns

(Index(['id', 'success', 'totalPayment', 'gasUsed', 'gasOverhead', 'trigger',
        'event', 'block', 'transaction_index', 'log_index', 'hash',
        'gasCostInETH', 'gasPrice', 'full_tx_gas_used',
        'gasCostInETH_with_chainlink_premium',
        'gasCostInETH_without_chainlink_overhead'],
       dtype='object'),
 Index(['destination', 'debtInfo', 'claimed', 'claimGasUsed', 'event', 'block',
        'transaction_index', 'log_index', 'hash', 'gas_price', 'gas_used',
        'gas_cost_in_eth'],
       dtype='object'),
 Index(['block', 'break_even_days', 'swapCost', 'apr_delta',
        'out_compositeReturn', 'in_compositeReturn',
        'predicted_increase_after_swap_cost',
        'predicted_gain_during_swap_cost_off_set_period', 'inEthValue',
        'outEthValue', 'out_destination', 'in_destination', 'offset_period',
        'slippage', 'hash', 'moveName', 'gasCostInETH'],
       dtype='object'))

In [ ]:
gas_cost_df = pd.concat([keeper_gas_costs_df[[]]])

In [37]:
keeper_gas_costs_df[["gasCostInETH_with_chainlink_premium"]]

,gasCostInETH_with_chainlink_premium
timestamp,
2024-09-15 00:58:47+00:00,0.003549
2024-09-15 02:58:59+00:00,0.001268
2024-09-15 03:10:23+00:00,0.005616
2024-09-15 03:12:35+00:00,0.001370
2024-09-15 03:59:11+00:00,0.001211
...,...
2024-10-29 12:33:11+00:00,0.023163
2024-10-29 13:30:35+00:00,0.011493
2024-10-29 13:33:23+00:00,0.021043


In [ ]:
keeper_gas_costs_df

,destination,debtInfo,claimed,claimGasUsed,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,,
2024-09-19 04:02:23+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,"(True, 0, 0, 414396248259951398280, 4143695129...",0,492350,DestinationDebtReporting,20782123,147,362,0x39a5a532b0f070668ab5076aa5717b607149c93a219c...
2024-09-19 04:02:23+00:00,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,"(True, 0, 0, 280688392880098872407, 2806897686...",0,396639,DestinationDebtReporting,20782123,147,364,0x39a5a532b0f070668ab5076aa5717b607149c93a219c...
2024-09-20 04:02:35+00:00,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,"(True, 0, 0, 280647818396993667637, 2806883928...",49341239610598684,674969,DestinationDebtReporting,20789288,196,695,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
2024-09-20 04:02:35+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,"(True, 0, 0, 750055284549229694347, 7500908019...",197333353426279214,369881,DestinationDebtReporting,20789288,196,699,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
2024-09-20 04:02:35+00:00,0xC001f23397dB71B17602Ce7D90a983Edc38DB0d1,"(True, 0, 0, 399231905705283421671, 3992000490...",0,353663,DestinationDebtReporting,20789288,196,701,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
...,...,...,...,...,...,...,...,...,...
2024-10-29 05:07:47+00:00,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9,"(True, 0, 0, 951916716893846004911, 9519176295...",0,353589,DestinationDebtReporting,21069102,120,373,0xdd97a7e72357e960fc453ace7c3ad326e5e271d0def1...
2024-10-29 05:07:47+00:00,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,"(True, 0, 0, 2930707357504115782099, 293140559...",264824222146590684,443890,DestinationDebtReporting,21069102,120,377,0xdd97a7e72357e960fc453ace7c3ad326e5e271d0def1...
2024-10-29 05:07:47+00:00,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,"(True, 0, 0, 2445007771553795252902, 244554876...",263969243919267956,331678,DestinationDebtReporting,21069102,120,381,0xdd97a7e72357e960fc453ace7c3ad326e5e271d0def1...


In [44]:
debt_reporting_df.tail(1)

,destination,debtInfo,claimed,claimGasUsed,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,,
2024-10-29 05:08:23+00:00,0x40219bBda953ca811d2D0168Dc806a96b84791d9,"(True, 0, 0, 668311549794773094131, 6683102437...",45506132673603222,573163,DestinationDebtReporting,21069105,141,398,0x34d8283a85c61682fd73af90c28a2426214246e8079b...


In [49]:
df = pd.concat(debt_reporting_dfs)
df["hash"].value_counts()

hash
0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc    24
0x862921210d71bb704a2090218715901800e2ed6b6e33573edb042d7c1aac1980    24
0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    20
0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    20
0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    20
                                                                      ..
0x890cb0576ae13d86a737aeb638cbc7e9aab43c5de66d8f39f2ff5b447c197a35     3
0x34d8283a85c61682fd73af90c28a2426214246e8079b77f663429baf4fe5b928     3
0x39a5a532b0f070668ab5076aa5717b607149c93a219c2fb9f6e1fca0396019ae     2
0x40ad773427a284900548755da8bd31a8e18ce3610907adeede023e3c53d547e3     2
0xaffcc4eec452011f8e08cc4e99dbf8bc4cd56e62e0106352c1dd00d5e6cf9069     2
Name: count, Length: 212, dtype: int64

In [50]:
df

,destination,debtInfo,claimed,claimGasUsed,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,,
2024-09-19 04:02:23+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,"(True, 0, 0, 414396248259951398280, 4143695129...",0,492350,DestinationDebtReporting,20782123,147,362,0x39a5a532b0f070668ab5076aa5717b607149c93a219c...
2024-09-19 04:02:23+00:00,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,"(True, 0, 0, 280688392880098872407, 2806897686...",0,396639,DestinationDebtReporting,20782123,147,364,0x39a5a532b0f070668ab5076aa5717b607149c93a219c...
2024-09-20 04:02:35+00:00,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,"(True, 0, 0, 280647818396993667637, 2806883928...",49341239610598684,674969,DestinationDebtReporting,20789288,196,695,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
2024-09-20 04:02:35+00:00,0xE382BBd32C4E202185762eA433278f4ED9E6151E,"(True, 0, 0, 750055284549229694347, 7500908019...",197333353426279214,369881,DestinationDebtReporting,20789288,196,699,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
2024-09-20 04:02:35+00:00,0xC001f23397dB71B17602Ce7D90a983Edc38DB0d1,"(True, 0, 0, 399231905705283421671, 3992000490...",0,353663,DestinationDebtReporting,20789288,196,701,0x62221b0f09863e8cf60f582dc1d26e44feef4aa1ad79...
...,...,...,...,...,...,...,...,...,...
2024-10-28 17:03:59+00:00,0x60339056EC88996e41757E05a798310E46972cca,"(True, 0, 0, 136483803881937211881, 1364838038...",0,377369,DestinationDebtReporting,21065502,1,54,0x1bde4145d38af2e66374d3170ca7847389ac365d9df6...
2024-10-28 17:03:59+00:00,0x40219bBda953ca811d2D0168Dc806a96b84791d9,"(True, 0, 0, 582768780210664559583, 5827687802...",0,385797,DestinationDebtReporting,21065502,1,56,0x1bde4145d38af2e66374d3170ca7847389ac365d9df6...
2024-10-29 05:08:23+00:00,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,"(True, 0, 0, 2139711998139967036974, 213953904...",73808444544482878,482561,DestinationDebtReporting,21069105,141,390,0x34d8283a85c61682fd73af90c28a2426214246e8079b...


In [52]:
debt_reporting_df[["hash"]].drop_duplicates()

,hash
timestamp,
2024-09-18 02:27:23+00:00,0xaffcc4eec452011f8e08cc4e99dbf8bc4cd56e62e010...
2024-09-19 02:27:23+00:00,0x799bc090bec9091593d4c0eef5f40e42372bef5af690...
2024-09-20 02:27:23+00:00,0x5b5b93b13fd76bf33341c64e841ff388467214fb03d5...
2024-09-21 02:27:23+00:00,0x0dd3b618668b805e1d8defabac49c22e41245f58a42f...
2024-09-21 12:08:35+00:00,0xa50e764980927b1fe3f4c348c8ab0770fc0e116e98dd...
...,...
2024-10-27 16:02:35+00:00,0x4fe0cc3294910e4dce949019259365ae3dffbffec99d...
2024-10-28 04:02:59+00:00,0x0bf7fa8689e0783b29ca56d46989a364a908a8903ee9...
2024-10-28 16:03:11+00:00,0xc0f350a04bf3d96b8cc10b9c0a284cdf10601abba763...


In [45]:
debt_reporting_df.tail(1).values

array([['0x40219bBda953ca811d2D0168Dc806a96b84791d9',
        (True, 0, 0, 668311549794773094131, 668310243725079682313, 668307301699180179033, 668305024649950316581, 668315797890366009229, 668315462800209048044),
        45506132673603222, 573163, 'DestinationDebtReporting', 21069105,
        141, 398,
        '0x34d8283a85c61682fd73af90c28a2426214246e8079b77f663429baf4fe5b928']],
      dtype=object)

In [ ]:
# 2,026,274

In [ ]:
def compute_expenses_from_gas() -> dict[str, dict[str, float]]:
    """
    Get the ETH value spent on gas to run the onchain part of tokemak over the last 7, 30 and 365 days

    - Solver gas cost (as of Oct 29, the solver does not take any profit)
    - Keeper Network Gas Costs

    """


def _get_gas_costs_of_solver() -> pd.DataFrame:
    #
    pass


def _get_gas_costs_of_reward_liqudation() -> pd.DataFrame:
    #
    pass

In [ ]:
def compute_protocol_level_profits():

    revenue = compute_revenue_from_fees()